<b> This code is designed to evaluated temperature with ENSO bins for the HAD data of ELI </b>

In [1]:
import xarray as xr
from metpy.constants import g, omega as om, Re as a
from metpy.units import units
import netCDF4
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Upload HAD the seasonally average anomalies for the HAD 

dir = '/chinook/dallmann/MPI/'
fn_had_anom = 'tas_MPI_1950-2014_anomDJF.nc'

data_had_his = xr.open_dataset(dir+fn_had_anom)

# Parse using MetPy, so we can use MetPy operations with it
#data_had_pr_his = data_had_pr_his.metpy.parse_cf()

# output directory and file
dro = '/chinook/dallmann/MPI/ELI_Bins/'
fn_output = 'tas_nino_moderate_MPI_anomDJFmean.nc'

In [3]:
had_newtime = data_had_his
#had_newtime['time'] = data_had_his.indexes['time'].to_datetimeindex(unsafe=True)

#had_newtime

In [4]:
# Upload the array for Nino_strong
dir = '/chinook/dallmann/ENSO_bins/'
fn_ENSO = 'MPI_nino_moderate_DJFmean.nc'

data_ENSO = xr.open_dataset(dir+fn_ENSO)
data_ENSO 

<xarray.Dataset>
Dimensions:  (year: 11)
Coordinates:
  * year     (year) int64 1955 1956 1961 1980 1983 1990 1993 1994 2006 2010 2011
Data variables:
    eli      (year) float64 ...

In [5]:
# Convert time to year so it can be found in an Integer64
# I am unsure if this is right since it already was seasonally averaged (unsure what to do about the .mean())
time_had = had_newtime.groupby('time.year').mean()
# shift the temp data by one year because cdo reconizes the year assoicated with december. 
time_had = time_had.shift(year=1)
                  
time_had

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 192, lon: 384, year: 66)
Coordinates:
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
    height    float64 2.0
  * year      (year) int64 1949 1950 1951 1952 1953 ... 2010 2011 2012 2013 2014
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (year, lon, bnds) float64 nan nan nan nan ... 358.6 358.6 359.5
    lat_bnds  (year, lat, bnds) float64 nan nan nan nan ... 88.82 88.82 89.75
    tas       (year, lat, lon) float32 nan nan nan nan ... 4.853 4.852 4.857

In [6]:
ts_nino_moderate = time_had.where(time_had['year'] == data_ENSO['year'], drop = True)
ts_nino_moderate

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 192, lon: 384, year: 11)
Coordinates:
  * year      (year) int64 1955 1956 1961 1980 1983 ... 1993 1994 2006 2010 2011
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
    height    float64 2.0
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (year, lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 359.5
    lat_bnds  (year, lat, bnds) float64 -89.75 -88.82 -88.82 ... 88.82 89.75
    tas       (year, lat, lon) float32 -0.8268 -0.8462 -0.8418 ... 4.205 4.213

In [7]:
# Create a new netcdf file for preciptiation relative to this respective year
ts_nino_moderate.to_netcdf(path=dro+fn_output)